In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
!pip install lightkurve --quiet

     |████████████████████████████████| 245kB 6.3MB/s 
     |████████████████████████████████| 6.5MB 8.3MB/s 
     |████████████████████████████████| 256kB 44.9MB/s 
     |████████████████████████████████| 22.3MB 1.5MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 3.2MB 37.4MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')  

Mounted at /content/drive


In [ ]:
book_local = 'shallue_local_curves_pt6'
book_global = 'shallue_global_curves_pt6'
path_input = "/content/drive/MyDrive/TCC/Base de Dados/lighkurve_KOI_dataset.csv"
path_local = '/content/drive/MyDrive/TCC/Experimentos/' + book_local + '.xlsx'
path_global = '/content/drive/MyDrive/TCC/Experimentos/' + book_global + '.xlsx'

import pandas as pd
import numpy as np
import time
from lightkurve import search_lightcurve 

lc = pd.read_csv(path_input, sep = ",") 
lc = lc[['kepid','koi_disposition','koi_period','koi_time0bk','koi_duration','koi_quarters']]

lc.shape
print('total inicial de curvas: %d\n'%(lc.shape[0]))

lc = lc.dropna()
lc = lc[lc.koi_disposition != 'CANDIDATE']
lc = lc.reset_index(drop=True)
print('falsos positivos: %d, confirmados: %d\n\ntotal atualizado: %d\n'%((lc.koi_disposition == 'FALSE POSITIVE').sum(),(lc.koi_disposition == 'CONFIRMED').sum(),lc.shape[0]))

perc_class = ((lc.koi_disposition == 'FALSE POSITIVE').sum()*100)/lc.shape[0]
print('falsos positivos: %.2f %% confirmados: %.2f %% \n'%(perc_class,100-perc_class))

import sys
import warnings
warnings.simplefilter("ignore")

curvas_locais = []
labels_locais = []
curvas_globais = []
labels_globais = []
start_time = time.time()
for index, row in lc[5000:6000].iterrows():
  period, t0, duration_hours = row[2], row[3], row[4]

  try:

    lcs = search_lightcurve(str(row[0]), author='Kepler', cadence='long').download_all()

    if (lcs != None):

      lc_raw = lcs.stitch()
      lc_raw.flux.shape

      lc_clean = lc_raw.remove_outliers(sigma=3)

      temp_fold = lc_clean.fold(period, epoch_time=t0)
      fractional_duration = (duration_hours / 24.0) / period
      phase_mask = np.abs(temp_fold.phase.value) < (fractional_duration * 1.5)
      transit_mask = np.in1d(lc_clean.time.value, temp_fold.time_original.value[phase_mask])

      lc_flat, trend_lc = lc_clean.flatten(return_trend=True, mask=transit_mask)

      lc_fold = lc_flat.fold(period, epoch_time=t0)

      #global preprocessing-----------------------------------------------------
      lc_global = lc_fold.bin(bins=2001).normalize() - 1
      lc_global = (lc_global / np.abs(np.nanmin(lc_global.flux)) ) * 2.0 + 1
      lc_global.flux.shape
      #global preprocessing-----------------------------------------------------
    
      phase_mask = (lc_fold.phase > -4*fractional_duration) & (lc_fold.phase < 4.0*fractional_duration)
      lc_zoom = lc_fold[phase_mask]

      #local preprocessing------------------------------------------------------
      lc_local = lc_zoom.bin(bins=201).normalize() - 1
      lc_local = (lc_local / np.abs(np.nanmin(lc_local.flux)) ) * 2.0 + 1
      lc_local.flux.shape
      #local--------------------------------------------------------------------

      labels_locais.append(row[1])
      curvas_locais.append(lc_local.flux.value)

      labels_globais.append(row[1])
      curvas_globais.append(lc_global.flux.value)

      print(index, 'OK')

    else:
      print(index, 'not downloaded')  
    
  except Exception as e:
    print(index, e)

t = time.time() - start_time   

print('Tempo para importar curvas de luz: %f seconds\n' %t)

dataset_global = pd.DataFrame(curvas_globais)
dataset_local = pd.DataFrame(curvas_locais)

for i in range(1,len(curvas_globais)):
    if len(curvas_globais[i]) != len(curvas_globais[i-1]):
        print("A curva %d possui tamanho diferente das demais curvas. Tamanho: %d"%(i,len(curvas_globais[i])))
print("Caso nenhuma das curvas apresente tamanho diferente, todas as curvas GLOBAIS possuem o total de %d pontos cada.\n"%len(curvas_globais[0]))

for i in range(1,len(curvas_locais)):
    if len(curvas_locais[i]) != len(curvas_locais[i-1]):
        print("A curva %d possui tamanho diferente das demais curvas. Tamanho: %d"%(i,len(curvas_locais[i])))
print("Caso nenhuma das curvas apresente tamanho diferente, todas as curvas LOCAIS possuem o total de %d pontos cada.\n"%len(curvas_locais[0]))

print("Quantidade de NaN na base GLOBAL: %s"%dataset_global.isna().sum(axis=1).sum())
print("Quantidade de NaN na base LOCAL: %s\n"%dataset_local.isna().sum(axis=1).sum())

perc_nan_glob = (dataset_global.isna().sum(axis=1).sum()*100)/dataset_global.count(axis=1).sum()
print("Porcentagem de valores do dataset GLOBAL substituídos na interpolação: %.2f %%"%perc_nan_glob)
perc_nan_loc = (dataset_local.isna().sum(axis=1).sum()*100)/dataset_local.count(axis=1).sum()
print("Porcentagem de valores do dataset LOCAL substituídos na interpolação: %.2f %%"%perc_nan_loc)

dataset_global = dataset_global.interpolate(axis=1)
dataset_local = dataset_local.interpolate(axis=1)

print("Quantidade de NaN na base GLOBAL após interpolação: %s --> deve sempre ser zero"%dataset_global.isna().sum(axis=1).sum())
print("Quantidade de NaN na base LOCAL após interpolação: %s --> deve sempre ser zero"%dataset_local.isna().sum(axis=1).sum())

labels_glob = pd.Series(labels_globais)
labels_loc = pd.Series(labels_locais)
dataset_global['label'] = labels_glob
dataset_local['label'] = labels_loc

dataset_global.to_csv(path_global,index=False)  
dataset_local.to_csv(path_local,index=False)

total inicial de curvas: 9564

falsos positivos: 4170, confirmados: 2349

total atualizado: 6519

falsos positivos: 63.97 % confirmados: 36.03 % 

5000 OK
5001 OK
5002 OK
5003 OK
5004 OK
5005 OK
5006 OK
5007 index 0 is out of bounds for axis 0 with size 0
5008 OK
5009 OK
5010 OK
5011 OK
5012 OK
5013 OK
5014 index 0 is out of bounds for axis 0 with size 0
5015 OK
5016 OK
5017 OK
5018 OK
5019 OK
5020 OK
5021 OK
5022 OK
5023 index 0 is out of bounds for axis 0 with size 0
5024 OK
5025 OK
5026 OK
5027 cannot reshape array of size 0 into shape (0,newaxis)
5028 OK
5029 cannot reshape array of size 0 into shape (0,newaxis)
5030 OK
5031 OK
5032 cannot reshape array of size 0 into shape (0,newaxis)
5033 OK
5034 OK
5035 OK
5036 index 0 is out of bounds for axis 0 with size 0
5037 OK
5038 OK
5039 OK
5040 OK
5041 OK
5042 index 0 is out of bounds for axis 0 with size 0
5043 OK
5044 OK
5045 OK
5046 OK
5047 index 0 is out of bounds for axis 0 with size 0
5048 index 0 is out of bounds for axis 0 with 

In [ ]:
pd.read_csv(path_global)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,label
0,0.906128,0.658511,1.047171,1.134320,0.761786,0.824519,0.389886,1.117720,1.103496,1.126642,0.959884,1.319620,1.349664,0.849160,1.094671,1.280412,0.839612,0.938350,0.670234,0.940968,1.047268,0.763760,1.051678,1.084204,1.132365,0.652841,0.727589,0.986947,1.188356,1.110603,0.913534,0.739287,1.392731,1.405832,0.670785,1.174254,0.821103,1.107792,0.867048,1.188937,...,1.394541,0.993062,1.041229,1.138973,0.870123,1.002968,1.107528,0.991933,0.935829,0.801090,1.212033,1.173105,1.225572,1.200073,0.913595,1.072911,0.580635,0.746321,1.006156,0.927490,0.953916,0.992694,0.976439,1.660159,0.664572,0.920214,1.061114,1.205646,0.671927,0.870725,1.165293,0.822024,0.867801,1.116436,0.919397,1.386711,0.866967,1.027353,0.801634,CONFIRMED
1,0.936686,1.281255,1.018259,1.020185,0.917401,1.139281,0.779179,1.042523,1.134919,1.475380,1.207334,1.145926,1.047984,0.969579,1.045859,1.044026,1.214894,0.955160,1.305320,0.810185,0.972808,0.521268,1.118013,1.040589,0.758880,0.929197,0.648613,0.972937,0.807056,0.842916,0.569152,1.087960,0.847783,0.892036,1.191895,1.296782,0.665237,1.025038,1.281863,0.743935,...,0.943035,0.892318,1.253170,1.133408,1.257960,0.942344,0.858842,1.261660,0.974457,0.773780,1.025995,1.090897,1.128336,0.950903,0.916134,0.996990,0.835894,0.804669,1.145469,0.460716,0.794620,0.900116,0.789553,1.234979,0.928472,1.043250,0.898910,1.105299,0.857212,0.938870,1.089399,0.912035,0.707878,1.207390,1.098815,1.496738,0.895630,1.067254,0.715531,CONFIRMED
2,1.117316,1.035651,1.027824,1.019998,1.012171,1.004344,1.043196,1.110710,1.149652,0.981709,0.954275,0.960787,0.894300,0.877539,0.973232,0.940326,1.065119,0.953164,1.119469,1.139408,1.130835,1.114074,1.308108,1.075526,1.000986,1.099054,1.174358,1.249662,1.324965,1.100938,1.125339,0.935671,0.964806,1.064326,0.796398,0.865658,0.935942,0.896981,1.049896,1.022333,...,1.220343,0.984732,1.115186,0.905467,0.850172,1.050002,1.056055,0.979979,1.050548,0.973656,1.081977,1.048284,0.947426,1.027338,1.024949,1.037970,1.011933,1.070147,1.183424,1.296702,1.409979,1.062102,1.258490,1.108684,0.968273,0.931137,1.202769,1.150437,1.138517,1.085279,1.237001,0.775763,1.058655,1.188372,1.039539,1.013024,0.920441,1.145162,1.084099,FALSE POSITIVE
3,0.872721,1.364519,1.245795,1.098970,1.340438,1.656152,1.303025,0.970271,1.677620,1.412327,0.809237,0.761963,0.771118,1.053852,1.238938,0.674329,1.275450,0.907327,1.208957,0.836317,0.940444,0.853713,0.735094,1.043344,1.451887,1.150131,0.764937,0.496663,1.245483,0.912436,0.883962,0.390758,1.123642,0.537043,1.336316,1.075193,0.832118,1.235502,0.313263,0.935995,...,0.813126,1.181010,1.263633,1.285015,1.089474,0.666992,1.248737,1.434996,0.980625,1.079200,0.925703,1.130476,0.736206,1.045278,1.060306,0.921663,1.122690,0.806131,1.313875,0.545471,1.018160,1.209650,0.541618,1.334731,1.419975,0.949734,1.225945,0.621284,1.110390,1.089772,0.753037,1.466657,0.795841,1.010556,1.333880,0.616668,0.816517,0.973417,1.087072,CONFIRMED
4,0.984464,1.003999,1.121386,1.011571,0.961754,0.889008,1.295879,0.761262,1.012354,0.979873,0.971767,1.056242,0.770919,0.699352,1.328139,0.990403,1.240438,1.012833,1.319981,1.266804,0.676153,0.829835,0.952271,1.318054,1.284861,1.022003,1.036977,1.136497,0.935060,0.929725,0.932463,0.851586,0.917902,1.055109,0.959178,0.998924,0.892543,0.829995,0.931266,0.893783,...,0.914205,0.865288,1.171300,1.304898,0.861742,0.754205,0.970747,0.900697,0.840467,1.046869,1.025737,1.326025,1.005666,1.197892,1.308447,1.051560,0.823870,0.950661,0.862455,1.199006,1.042524,0.831266,0.944496,0.850624,1.038780,0.913328,0.867137,0.961273,0.784332,1.074923,1.040474,0.878411,1.255083,1.345634,1.088568,0.987708,0.983834,1.188810,0.968019,CONFIRMED
...,...,...,...,...,...,...,...,...,...,...,

In [ ]:
pd.read_csv(path_local)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,label
0,1.785769,2.729798,2.548395,3.248466,1.325620,1.023493,1.503760,0.328128,1.491723,-0.609683,0.875362,0.099179,2.318450,1.255574,1.655993,-0.090418,3.746318,0.955199,1.818295,1.526516,1.804322,0.643232,1.331742,0.977690,0.693099,0.762290,0.911264,0.633709,0.984837,0.528714,1.820707,1.050375,1.493809,2.138134,1.176128,0.247839,1.657595,0.426097,0.684143,1.034590,...,1.433007,0.978647,1.109933,2.463807,1.494053,1.398269,1.419461,0.402667,1.199391,0.292239,1.393902,0.499244,1.016115,2.950302,-0.045989,2.868798,1.405077,-0.352503,0.113811,0.703237,0.683633,1.540616,0.819415,1.216532,2.289819,3.299144,2.467030,1.634916,1.691216,1.145400,2.121168,2.081381,0.615004,2.437645,2.094423,0.313795,2.729227,1.091718,2.807821,CONFIRMED
1,0.587338,0.245809,-0.095720,-0.437250,-0.778779,-0.447903,-0.117028,0.213847,0.544723,0.875598,1.206474,1.537349,1.868224,1.456464,1.044704,0.632943,1.644274,1.643207,1.642140,1.641072,1.640005,1.638938,1.637871,1.636804,1.635737,1.471003,1.306270,1.141536,0.976802,0.812069,0.647335,0.482602,0.317868,0.153135,-0.011599,-0.176332,-0.341066,-0.505799,-0.670533,-0.835266,...,1.735280,1.753959,1.772638,1.791316,1.809995,1.663050,1.516104,1.369158,2.700903,2.504858,2.308814,2.112769,1.916724,1.720679,1.524635,1.328590,1.132545,1.149847,1.167148,1.184449,1.201751,1.219052,1.236353,1.253655,1.270956,1.288257,1.305559,1.322860,1.340161,1.357463,1.374764,1.392065,1.409367,1.409367,1.409367,1.409367,1.409367,1.409367,1.409367,CONFIRMED
2,0.995033,0.903242,0.941117,1.047910,1.015761,0.997172,0.965115,0.973159,0.364509,0.894915,1.072987,1.058085,1.095463,0.960293,0.896428,0.966828,1.048868,0.488940,1.000000,0.968325,0.931663,1.061310,0.840722,1.055096,0.939088,1.044877,0.922050,1.004848,0.938288,0.936956,1.023005,1.018915,0.932939,1.007516,0.950201,0.613911,1.052581,0.909249,1.069499,0.960075,...,1.021272,1.069684,0.978216,1.067139,1.054313,0.977043,0.989317,1.001591,1.088075,1.010980,1.113904,1.142468,1.035748,1.095366,1.094858,1.100043,1.105227,1.009047,1.088413,0.911689,1.121987,1.110807,0.984893,1.124055,1.093078,1.062101,1.086826,1.152746,1.051311,1.018101,1.134081,1.116810,1.125367,0.679383,1.129905,1.023179,1.082135,1.044991,1.151153,FALSE POSITIVE
3,0.951343,1.128255,0.687507,1.486403,0.600641,1.163747,1.748795,1.489461,1.000000,1.079301,0.900568,1.136637,1.151875,0.779864,1.175867,1.454183,1.163356,1.344862,1.390686,1.501901,1.467314,1.352926,1.063316,1.011140,1.979916,0.881447,1.144565,1.265298,0.810284,1.150091,1.760274,1.227167,1.430559,1.415720,1.722104,0.871094,0.952616,0.989656,0.555363,1.017661,...,1.449911,0.880370,1.499635,1.330331,1.106349,1.252300,1.027485,1.338622,1.856384,1.683449,1.282409,1.486427,0.448646,1.108171,1.146930,1.557702,0.953019,1.888019,0.936058,1.383900,0.852225,1.184184,1.259360,0.842324,1.940689,1.194531,0.736479,0.906610,1.470937,1.313213,1.137279,0.926430,1.902571,1.249999,1.338689,0.468365,1.269225,1.805658,1.001325,CONFIRMED
4,0.139141,1.121875,3.047826,1.230143,0.240182,3.098975,0.503085,0.703715,2.584631,2.807380,0.731353,3.187248,3.377878,3.568509,0.280295,-0.831913,1.417438,0.496445,0.096400,1.417832,1.283800,0.925058,0.474172,1.000000,1.081579,1.654303,3.534544,3.958425,0.741869,0.190716,-0.533327,0.363024,1.708764,1.063962,1.053753,2.511500,1.163183,0.882988,1.334600,-0.866960,...,1.082818,0.824469,0.810959,-0.517491,2.163915,2.144454,0.094529,0.343628,1.316699,0.781447,2.386419,1.467646,1.776157,1.437403,0.921439,2.549227,-0.661642,1.204332,4.003225,1.091468,0.901160,0.710852,1.012172,0.934386,0.942124,-0.606805,1.724989,1.783951,7.516794,1.339558,1.014254,0.688949,2.474684,1.269588,1.697543,0.624603,0.100657,0.789521,1.478385,CONFIRMED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..